<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/1-4-LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Low-Rank Adaptation (LoRA)은 대형 신경망, 특히 트랜스포머 모델의 미세 조정(fine-tuning) 과정에서 훈련해야 하는 파라미터 수를 줄이기 위해 사용되는 기법입니다. 이는 가중치 행렬을 저순위(low-rank) 행렬로 분해하여 계산 부담을 줄이는 방식으로 이루어집니다.

아래는 PyTorch를 사용하여 LoRA를 단일 선형 계층에 적용하는 기본 예제입니다.

In [1]:
import torch
import torch.nn as nn

class LoRALinear(nn.Module):
    def __init__(self, in_features, out_features, rank=4):
        super(LoRALinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.rank = rank

        # 기존 가중치
        self.weight = nn.Parameter(torch.randn(out_features, in_features))

        # 저순위 분해 행렬들
        self.lora_A = nn.Parameter(torch.randn(rank, in_features))
        self.lora_B = nn.Parameter(torch.randn(out_features, rank))

        # 초기 LoRA 레이어의 효과가 작도록 보장하는 스케일링 인자
        self.scaling = 1.0 / rank

    def forward(self, x):
        # 기존 선형 계층의 계산
        output = torch.matmul(x, self.weight.t())

        # LoRA 조정
        lora_adjustment = torch.matmul(x, self.lora_A.t())
        lora_adjustment = torch.matmul(lora_adjustment, self.lora_B.t())

        # 원래 출력에 LoRA 조정을 더함
        output = output + self.scaling * lora_adjustment
        return output

# 사용 예시
if __name__ == "__main__":
    # LoRA가 적용된 선형 계층 정의
    lora_layer = LoRALinear(in_features=128, out_features=64, rank=4)

    # 입력 텐서 예제 (128개의 특징을 가진 10개의 샘플로 구성된 배치)
    x = torch.randn(10, 128)

    # LoRA가 적용된 선형 계층을 통한 순전파 계산
    output = lora_layer(x)

    print(f"Output shape: {output.shape}")


Output shape: torch.Size([10, 64])


LoRALinear 클래스:

in_features와 out_features: 이 값들은 선형 계층의 입력 및 출력 차원을 정의합니다.
기존 가중치 행렬: 일반적인 선형 계층에서 사용하는 가중치 행렬(self.weight)입니다.
LoRA 분해:
lora_A: [rank, in_features] 형태의 저순위 행렬입니다.
lora_B: [out_features, rank] 형태의 또 다른 저순위 행렬입니다.
스케일링 인자: LoRA의 초기 효과가 작도록 하여 훈련의 안정성을 높입니다.
순전파 (Forward Pass):

순전파 계산은 self.weight를 사용하여 일반적인 선형 변환을 계산하는 것에서 시작합니다.
이후, LoRA 조정은 lora_A로 x를 변환한 후, 결과를 lora_B로 다시 변환하여 계산됩니다.
최종 출력은 기존 변환 결과에 스케일링된 LoRA 조정을 더한 값이 됩니다.
사용법:

LoRALinear 클래스는 일반적인 nn.Linear 계층처럼 사용됩니다. 이 예제에서는 입력 텐서 x가 LoRA가 적용된 선형 계층을 통과합니다.
LoRA의 장점:
파라미터 효율성: LoRA는 전체 가중치 행렬을 직접 훈련하는 대신 저순위 행렬을 사용하여 훈련해야 하는 파라미터 수를 줄입니다.
메모리 및 계산 효율성: 파라미터 수가 줄어들면 메모리 사용량과 계산 요구 사항이 줄어들어 대형 모델을 미세 조정하기가 쉬워집니다.
이 예제는 LoRA의 개념을 설명하기 위한 간단한 예시입니다. 실제로는 더 복잡한 모델(예: 트랜스포머)에 적용되며, 더 정교한 스케일링 및 정규화 기법이 포함될 수 있습니다.